In [3]:
import os
import requests                      # HTTP client for API calls
import pandas as pd                  # Tabular data handling
from datetime import datetime        # Datetime handling
from typing import Iterable, Optional, Dict, Union
import matplotlib.pyplot as plt
%pip install yfinance
import yfinance as yf
from src.debug_print import debug_print

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------- ----------------------------- 0.8/3.0 MB 4.8 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 5.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.6 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.8 MB/s eta 0:00:00
  Created wheel for multitasking: filename

In [8]:
dict = {
    'date': [
        '2024-06-03',  
        '2024-06-04',
        '2024-06-05',
        '2024-06-06',  
        '2024-06-07',
        ],
    'AAPL': [
        192.721024,
        192.621024,
        192.521024,
        192.521024,
        192.421024
        ]
        }
df = pd.DataFrame(dict)
print(df)

second_level = [
    "fiat",
    "crypto",
]

df.columns = pd.MultiIndex.from_arrays(
    [df.columns, second_level],
    names=["rate", "asset_type"],
)
print(df)

         date        AAPL
0  2024-06-03  192.721024
1  2024-06-04  192.621024
2  2024-06-05  192.521024
3  2024-06-06  192.521024
4  2024-06-07  192.421024
rate              date        AAPL
asset_type        fiat      crypto
0           2024-06-03  192.721024
1           2024-06-04  192.621024
2           2024-06-05  192.521024
3           2024-06-06  192.521024
4           2024-06-07  192.421024
